In [90]:
import requests
from datetime import datetime
from bs4 import BeautifulSoup
from math import nan
import json, re, os, time

from selenium.webdriver import Chrome
import chromedriver_binary

headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36"}

def render_html(url):
    req = requests.get(url,headers=headers)
    return BeautifulSoup(req.text,"html.parser")

In [91]:
url = "https://timingjobs.pl/"
page = render_html(url)

In [92]:
containers = []
for i in range(1,7):
    url = "https://timingjobs.pl/jm-ajax/get_listings/"
    payload = {
        "lang": "",
        "search_categories": "",
        "search_keywords": "",
        "search_location": "",
        "filter_post_status": "publish",
        "per_page": 6,
        "orderby": "featured",
        "order": "desc",
        "page": i,
        "show_pagination": False,
    }
    data = requests.post(url,headers=headers,data=json.dumps(payload)).json()
    li_s = data['html']    
    containers.append(li_s)
    print(i,end=",")
print("\b.")

1,2,3,4,5,6.


In [93]:
jobs = []
for container in containers:
    tag = BeautifulSoup(container.strip(),"html.parser")
    for li in tag.find_all("li",class_="job_listing"):
        link = li.find("a")['href']
        title = li['data-title']
        company = li.find("img")['alt']
        location = li.find("div",{"class":"location"}).get_text().strip()
        wage = li.find("li",{"class":"job-type"}).get_text().strip()
        date = li.find("time")['datetime']
        job = {
            "Link": link,
            "Title": title,
            "Company": company,
            "Location": location,
            "Wage": wage,
            "Date": date,
        }
        jobs.append(job)

In [94]:
jobs

[{'Link': 'https://timingjobs.pl/oferta/pracownik-magazynu-logistycznego/',
  'Title': 'Pracownik magazynu logistycznego',
  'Company': 'GLS',
  'Location': 'Wijchen',
  'Wage': '11.85€/h',
  'Date': '2022-02-07'},
 {'Link': 'https://timingjobs.pl/oferta/spawacz-mig-mag/',
  'Title': 'SPAWACZ MIG/MAG',
  'Company': '',
  'Location': 'Gdziekolwiek',
  'Wage': '15€/h',
  'Date': '2022-02-07'},
 {'Link': 'https://timingjobs.pl/oferta/order-picker-pracownik-magazynu/',
  'Title': 'Order picker – Pracownik magazynu',
  'Company': 'Arvato',
  'Location': 'Heijen',
  'Wage': '10.75€/h',
  'Date': '2022-01-31'},
 {'Link': 'https://timingjobs.pl/oferta/operator-wozka-widlowego-2/',
  'Title': 'Operator wózka widłowego',
  'Company': 'Ceva Logistics',
  'Location': 'Venray',
  'Wage': '12.14€/h',
  'Date': '2022-01-31'},
 {'Link': 'https://timingjobs.pl/oferta/pracownik-szklarni-gipmans/',
  'Title': 'Pracownik szklarni',
  'Company': 'Gipmans',
  'Location': 'Venlo',
  'Wage': '10.48€/h',
  'Da